In [ ]:
from datetime import timedelta
import streamlit as st
import snowflake.snowpark.functions as F
import altair as alt

## Visualize Daily Sales on Snowsight

Before building our model, let's first visualize our data to get a feel for what daily sales looks like.

### View Sample Data

In [ ]:
session  = get_active_session()
mncheese_sales_df = session.table("tasty_byte_sales").where(F.col("menu_item_name").like('Lobster Mac & Cheese'))
mncheese_sales_df.show()

### Plot the data
Let us now plot our sales data using Streamlit

In [ ]:
base = alt.Chart(mncheese_sales_df.to_pandas())

total_sold = base.mark_line(point=True).encode(
    x=alt.X("DATE:O", timeUnit="monthdate", title="Date").scale(zero=False),
    y=alt.Y("TOTAL_SOLD", title="Total Sold").scale(zero=False),
)
st.altair_chart(total_sold,use_container_width=True)

## Build Forecast Model

### Dataset

Let us build the sales dataset for the city of `vancouver` with past 1 year's data. We will persist the data on to a table named `vancouver_sales`

In [ ]:
date_range = session.table("tasty_byte_sales").select(F.max(F.col("date")).alias("max_date"))
max_d = date_range.first()[0]
# print(f"Current Date:{max_d} and its type {type(max_d)}")
interval = max_d - timedelta(365)
# print(f"Interval: {interval}")

van_sales = session.table("tasty_byte_sales").select(F.to_timestamp_ntz("date").alias("timestamp"),F.col("primary_city"),F.col("menu_item_name"),F.col("total_sold")).where(F.col("date") > interval)
# print(van_sales.queries)
van_sales.write.mode("overwrite").save_as_table("vancouver_sales")

### Create Timeseries Data

The forecasting model relies on timeseries data. Let us create a view to build a timesries data from sales data `vancouver_sales` and filter it for `Lobster Mac & Cheese`.

In [ ]:
CREATE OR REPLACE VIEW lobster_sales AS (
    SELECT
        timestamp,
        total_sold
    FROM
        vancouver_sales
    WHERE
        menu_item_name LIKE 'Lobster Mac & Cheese'
);

### Build a Forecast Model

In [ ]:
CREATE OR REPLACE forecast lobstermac_forecast (
    INPUT_DATA => SYSTEM$REFERENCE('VIEW', 'lobster_sales'),
    TIMESTAMP_COLNAME => 'TIMESTAMP',
    TARGET_COLNAME => 'TOTAL_SOLD'
);

### List all models

In [ ]:
SHOW forecast;

### Forecast

Let us use the model to build a sales forecast for next 10 days. And persist the predictiosn on to a table named `macncheese_predictions`.

In [ ]:
pred_df = session.sql("""
    CALL lobstermac_forecast!FORECAST(FORECASTING_PERIODS => 10)
    """).write.mode("overwrite").save_as_table("macncheese_predictions")

### Visualize the Forecast

Let us build a visualization of the forecast.


In [ ]:
sales_df = session.table("lobster_sales").select(F.col("timestamp"),F.col("total_sold")).with_column("forecast",F.lit(None)).where(F.col('timestamp') > '2023-03-01')

forecast_df = session.table("macncheese_predictions").select(F.col("TS").alias("timestamp"),F.col("forecast")).with_column("total_sold",F.lit(None))
df = sales_df.union_all_by_name(forecast_df)

In [ ]:
df2 = df.to_pandas()
base = alt.Chart(df2)

total_sold = base.mark_line(point=True, color="blue").encode(
    x=alt.X("TIMESTAMP:O",timeUnit="monthdate",title="Date").scale(zero=False),
    y=alt.Y("TOTAL_SOLD", title="Total Sold").scale(zero=False),
)

forecast = base.mark_line(point=True, color="orange").encode(
    x=alt.X("TIMESTAMP:O",timeUnit="monthdate",title="Date"),
    y=alt.Y(shorthand="FORECAST", title="Total Sold").scale(zero=False),
)
st.altair_chart(total_sold + forecast, use_container_width=True)

In [ ]:
CALL lobstermac_forecast!FORECAST(FORECASTING_PERIODS => 10, CONFIG_OBJECT => {'prediction_interval': .5});